# Using Structured Output

[Structured Output](https://dev.writer.com/home/structured-output) allows you to specify a response schema for chat completions with Palmyra LLMs. With structured output, you can receive predictable, machine-readable responses from the AI.

This cookbook shows how to use structured output to enforce response formats, whether using JSON Schema or Pydantic objects, and demonstrates both streaming and non-streaming methods.

## When to use structured output

- When you need responses in a **specific format** (e.g., recipes, movie recommendations, product details).  
- When responses will be **parsed by code**, avoiding free-text ambiguity.  
- When integrating AI outputs into **databases, APIs, or dashboards**.  

## Prerequisites

Before getting started, you'll need:

- A [Writer AI Studio](https://app.writer.com/register) account
- An API key, which you can obtain by following the [API Quickstart](https://dev.writer.com/api-guides/quickstart)
- Familiarity with either [JSON Schema](https://json-schema.org/) or [Pydantic](https://docs.pydantic.dev/latest/) for defining structured data
- The Writer SDK installed (see Setup below)

## Setup

Install the Writer SDK:

In [ ]:
%pip install -qU writer-sdk

Next, set the `WRITER_API_KEY` environment variable. We recommend setting it in a `.env` file in the root of your project, but this tutorial will set it in an environment variable if you don't have a `.env` file.

In [ ]:
import getpass
import os
import json
from typing import List
from pydantic import Field, BaseModel
from writerai import Writer

if not os.getenv("WRITER_API_KEY"):
    os.environ["WRITER_API_KEY"] = getpass.getpass("Enter your Writer API key: ")

client = Writer()  # The Writer client automatically uses the WRITER_API_KEY environment variable to authenticate.

## Usage

The following code snippet shows how to use structured output to enforce a specific response format, using a JSON Schema to define the structure of the response. It specifically extracts movie recommendations from a user query and returns them in a JSON object that matches the schema.


In [ ]:
system_message = "You are a helpful movie recommendation assistant."
user_message = "Please recommend the most interesting movie in the world."

response_format = {
        "type": "json_schema",
        "json_schema": {
            "schema": {
                "type": "object",
                "properties": {
                    "title": {"type": "string"},
                    "genre": {"type": "string"},
                    "year": {"type": "number"},
                    "rating": {"type": "number"},
                    "streaming_platforms": {"type": "array", "items": {"type": "string"}}
                },
                "required": ["title", "genre", "year", "rating", "streaming_platforms"]
            }
        }
    }

In [ ]:
response = client.chat.chat(
    model="palmyra-x5",
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ],
    response_format=response_format
)

response_json = json.loads(response.choices[0].message.content)
print(f"Recommended movie: {response_json['title']}")
print(response_json)

## Using Pydantic Objects and `chat.parse`

Instead of defining a `JSON` Schema manually, you can use [Pydantic models](https://docs.pydantic.dev/latest/) to define the structure of the AI’s response. The `chat.parse` helper validates and converts the response into a Python object.


In [ ]:
# Define a Pydantic model for movie recommendations
class MovieRecommendation(BaseModel):
    title: str = Field(description="The title of the movie")
    genre: str = Field(description="The genre or category of the movie")
    year: int = Field(description="The release year of the movie")
    rating: float = Field(description="The rating of the movie (e.g., IMDb score)")
    streaming_platforms: List[str] = Field(description="A list of streaming platforms where the movie is available")


parsed_response = client.chat.parse(
    model="palmyra-x5",
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ],
    response_format=MovieRecommendation
)

movie_recommendation = parsed_response.choices[0].message.parsed
print(f"Recommended movie: {movie_recommendation.title}")
print(movie_recommendation)

## Streaming
You can also stream the chat response, which will stream the response as it is generated. The following code snippet shows how to stream the response, collect the text as it is generated, and print it out.

In [ ]:
response_stream = client.chat.chat(
    model="palmyra-x5",
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ] ,
    response_format=response_format,
    stream=True
)


output_text = ""

for chunk in response_stream:
    if chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content, end="", flush=True)
        output_text += chunk.choices[0].delta.content


print(f"Final output: {output_text}")